# VacationPy Analysis

In [1]:
# Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

In [2]:
# Define a path to the csv file exported in Part 1
csvpath = '../WeatherPy/Weather_Output/cities.csv'

In [3]:
# Load the csv to a Pandas DataFrame
cities_df = pd.read_csv(csvpath)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,geraldton,-28.7667,114.6000,66.61,52.0,22.0,11.50,AU,2022-04-23
1,ushuaia,-54.8000,-68.3000,46.06,66.0,75.0,9.22,AR,2022-04-23
2,dodge city,37.7528,-100.0171,78.69,16.0,0.0,23.02,US,2022-04-23
3,mataura,-46.1927,168.8643,51.48,97.0,100.0,7.43,NZ,2022-04-23
4,bluff,-46.6000,168.3333,51.80,74.0,100.0,20.71,NZ,2022-04-23


## Humidity Heatmap

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

In [5]:
# Create a variable, locations, to store 'Lat' and 'Lng' 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Create a variable, humidity, to store city humidity values
humidity = cities_df['Humidity'].astype(float)

In [6]:
# Create a Heatmap using latitude and longitude for the location of each city and humidty as the weight
# Configure the map
fig = gmaps.figure(center=(5,0), zoom_level=2, layout={'width': '975px','height': '700px',
                                                        'padding': '3px','border': '1px solid black'})

# Create the heat layer and add specifications
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 2.5)

# Add the heat layer to the map
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', padding='3px', width='975px'))

## Vacation Hotels based on Ideal Weather Conditions

In [7]:
# Create a new dataframe based on desired weather criteria for vacation
hotel_df = cities_df.loc[(cities_df['Max Temp'] > 80) & (cities_df['Max Temp'] <= 90) 
                                 & (cities_df['Humidity'] <= 70) & (cities_df['Cloudiness'] <= 10) 
                                 & (cities_df['Wind Speed'] <= 10)]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,govindgarh,27.5000,76.9833,85.17,13.0,1.0,4.07,IN,2022-04-23
59,sunam,30.1333,75.8000,80.35,20.0,0.0,6.87,IN,2022-04-23
146,cabo san lucas,22.8909,-109.9124,81.70,67.0,0.0,9.22,MX,2022-04-23
170,rock falls,41.7798,-89.6890,82.40,51.0,0.0,8.05,US,2022-04-23
186,hostotipaquillo,21.0667,-104.0667,88.05,6.0,0.0,7.65,MX,2022-04-23
204,arraial do cabo,-22.9661,-42.0278,87.76,58.0,0.0,6.91,BR,2022-04-23
439,eatonton,33.3268,-83.3885,82.24,56.0,3.0,9.24,US,2022-04-23
533,atar,20.5169,-13.0499,80.46,22.0,0.0,9.71,MR,2022-04-23
549,russell,32.3502,-85.1999,82.00,39.0,0.0,8.05,US,2022-04-23


In [8]:
# Add a blank 'Hotel Name' column to the DataFrame
hotel_df['Hotel Name'] = ''
hotel_df

/Users/graceolson/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,govindgarh,27.5000,76.9833,85.17,13.0,1.0,4.07,IN,2022-04-23,
59,sunam,30.1333,75.8000,80.35,20.0,0.0,6.87,IN,2022-04-23,
146,cabo san lucas,22.8909,-109.9124,81.70,67.0,0.0,9.22,MX,2022-04-23,
170,rock falls,41.7798,-89.6890,82.40,51.0,0.0,8.05,US,2022-04-23,
186,hostotipaquillo,21.0667,-104.0667,88.05,6.0,0.0,7.65,MX,2022-04-23,
204,arraial do cabo,-22.9661,-42.0278,87.76,58.0,0.0,6.91,BR,2022-04-23,
439,eatonton,33.3268,-83.3885,82.24,56.0,3.0,9.24,US,2022-04-23,
533,atar,20.5169,-13.0499,80.46,22.0,0.0,9.71,MR,2022-04-23,
549,russell,32.3502,-85.1999,82.00,39.0,0.0,8.05,US,2022-04-23,


In [9]:
# Set a base URL for Google Places API and parameters to search for hotels within 5,0000 meters of desired cities
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
params = {'type' : 'hotel',
          'radius' : 5000,
          'key' : g_key}

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    #Set a variable for retriving city coordinates from the DataFrame
    coords = f'{row["Lat"]},{row["Lng"]}'
    
    # Add location to parameters dictionary
    params['location'] = coords
    
    # Set a variable for retrieving the city name from the DataFrame
    city = hotel_df['City']
    
    # Assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']
    
    # Find the second hotel from the results (the first hotel for a lot of my locations was the name of the city itself, so I selected the second hotel from the results for more interesting data)
    # Store results in the 'Hotel Name' column of the DataFrame
    try:
        print(f"Closest hotel in {row['City']} is {results[1]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[1]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 23: govindgarh.
Closest hotel in govindgarh is Khandelwal Auto Parts.
------------
Retrieving Results for Index 59: sunam.


/Users/graceolson/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel in sunam is Shree Shiv Niketan Dharmshala,sunam.
------------
Retrieving Results for Index 146: cabo san lucas.
Closest hotel in cabo san lucas is Hotel Tesoro Los Cabos.
------------
Retrieving Results for Index 170: rock falls.
Closest hotel in rock falls is U-Haul Neighborhood Dealer.
------------
Retrieving Results for Index 186: hostotipaquillo.
Closest hotel in hostotipaquillo is Bansefi.
------------
Retrieving Results for Index 204: arraial do cabo.
Closest hotel in arraial do cabo is Pousada Porto Praia.
------------
Retrieving Results for Index 439: eatonton.
Closest hotel in eatonton is Subway.
------------
Retrieving Results for Index 533: atar.
Closest hotel in atar is Oumou Elghoura.
------------
Retrieving Results for Index 549: russell.
Closest hotel in russell is SHON & MANDY.
------------


In [10]:
# Display the new DataFrame with the hotels found using the Google Places API
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,govindgarh,27.5000,76.9833,85.17,13.0,1.0,4.07,IN,2022-04-23,Khandelwal Auto Parts
59,sunam,30.1333,75.8000,80.35,20.0,0.0,6.87,IN,2022-04-23,"Shree Shiv Niketan Dharmshala,sunam"
146,cabo san lucas,22.8909,-109.9124,81.70,67.0,0.0,9.22,MX,2022-04-23,Hotel Tesoro Los Cabos
170,rock falls,41.7798,-89.6890,82.40,51.0,0.0,8.05,US,2022-04-23,U-Haul Neighborhood Dealer
186,hostotipaquillo,21.0667,-104.0667,88.05,6.0,0.0,7.65,MX,2022-04-23,Bansefi
204,arraial do cabo,-22.9661,-42.0278,87.76,58.0,0.0,6.91,BR,2022-04-23,Pousada Porto Praia
439,eatonton,33.3268,-83.3885,82.24,56.0,3.0,9.24,US,2022-04-23,Subway
533,atar,20.5169,-13.0499,80.46,22.0,0.0,9.71,MR,2022-04-23,Oumou Elghoura
549,russell,32.3502,-85.1999,82.00,39.0,0.0,8.05,US,2022-04-23,SHON & MANDY


In [11]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Plot hotel markers on top of the heatmap
# Configure the map
fig = gmaps.figure(center=(5,0), zoom_level=2, layout={'width': '975px','height': '700px',
                                                        'padding': '3px','border': '1px solid black'})

# Create the markers layer and add specifications
hotel_layer = gmaps.marker_layer(hotel_locations, info_box_content = hotel_info)

# Add the heat and hotel layers to the map
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', padding='3px', width='975px'))